# Load Libraries

In [2]:
try:
    import pandas as pd
    import findspark
    findspark.init()

    
    from pyspark.sql import SparkSession
    from pyspark.context import SparkContext
    import pyspark.sql.functions
    from pyspark.sql.functions import *
    import  pyspark.sql.types
    from datetime import date ,timedelta ,datetime
    import time
    
    print("Successfully loaded Spark and Pandas ... ")
    
except ImportError as IE:
    print("Error loading Spark and Pandas ... ")
    print("This module was not installed {}".format())
    exit(1)

Successfully loaded Spark and Pandas ... 


# Initialize spark session

In [3]:
sc=SparkSession.builder.appName("PysparkExample2").getOrCreate()

# Read Data 

In [4]:
try:
    #we access the datasets and load them to memory as dataframes
    # items_data=sc.read.csv("../data/olist_order_items_dataset.csv",header=True,inferSchema=True)
    items_data=sc.read.format("csv").option("header","true").load("../data/olist_order_items_dataset.csv")
    #orders_data=sc.read.csv("../data/olist_orders_dataset.csv",header=True)
    orders_data=sc.read.format("csv").option("header","true").load("../data/olist_orders_dataset.csv")
    #products_data=sc.read.csv("../data/olist_products_dataset.csv",header=True)
    products_data=sc.read.format("csv").option("header","true").load("../data/olist_products_dataset.csv")
    
    print("Successfully loaded dataframes to memory ...")
except FileNotFoundError as fnf:
    print("The file {} you are looking for doesn't exist in that directory.".format(fnf))
    exit(1)

Successfully loaded dataframes to memory ...


In [5]:
items_data.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- order_item_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- shipping_limit_date: string (nullable = true)
 |-- price: string (nullable = true)
 |-- freight_value: string (nullable = true)



In [6]:
items_data.show(5)

+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date| price|freight_value|
+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|00010242fe8c5a6d1...|            1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 09:45:35| 58.90|        13.29|
|00018f77f2f0320c5...|            1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-05-03 11:05:13|239.90|        19.93|
|000229ec398224ef6...|            1|c777355d18b72b67a...|5b51032eddd242adc...|2018-01-18 14:48:30|199.00|        17.87|
|00024acbcdf0a6daa...|            1|7634da152a4610f15...|9d7a1d34a50524090...|2018-08-15 10:10:18| 12.99|        12.79|
|00042b26cf59d7ce6...|            1|ac6c3623068f30de0...|df560393f3a51e745...|2017-02-13 13:57:51|199.90|        18.14|
+--------------------+-------------+----

In [7]:
orders_data.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: string (nullable = true)
 |-- order_approved_at: string (nullable = true)
 |-- order_delivered_carrier_date: string (nullable = true)
 |-- order_delivered_customer_date: string (nullable = true)
 |-- order_estimated_delivery_date: string (nullable = true)



In [8]:
orders_data.show(5)

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

# Simple Queries

In [9]:
products_data.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: string (nullable = true)
 |-- product_description_lenght: string (nullable = true)
 |-- product_photos_qty: string (nullable = true)
 |-- product_weight_g: string (nullable = true)
 |-- product_length_cm: string (nullable = true)
 |-- product_height_cm: string (nullable = true)
 |-- product_width_cm: string (nullable = true)



## Select query

In [10]:
products_data.select("product_category_name","product_id").show(5)

+---------------------+--------------------+
|product_category_name|          product_id|
+---------------------+--------------------+
|           perfumaria|1e9e8ef04dbcff454...|
|                artes|3aa071139cb16b67c...|
|        esporte_lazer|96bd76ec8810374ed...|
|                bebes|cef67bcfe19066a93...|
| utilidades_domest...|9dc1a7de274444849...|
+---------------------+--------------------+
only showing top 5 rows



## When query

In [11]:
products_data.select("product_category_name","product_id",when(products_data['product_category_name']=='artes',1).otherwise(0)).show(5)

+---------------------+--------------------+-----------------------------------------------------------+
|product_category_name|          product_id|CASE WHEN (product_category_name = artes) THEN 1 ELSE 0 END|
+---------------------+--------------------+-----------------------------------------------------------+
|           perfumaria|1e9e8ef04dbcff454...|                                                          0|
|                artes|3aa071139cb16b67c...|                                                          1|
|        esporte_lazer|96bd76ec8810374ed...|                                                          0|
|                bebes|cef67bcfe19066a93...|                                                          0|
| utilidades_domest...|9dc1a7de274444849...|                                                          0|
+---------------------+--------------------+-----------------------------------------------------------+
only showing top 5 rows



In [12]:
products_data.describe().show()

+-------+--------------------+---------------------+-------------------+--------------------------+------------------+------------------+------------------+------------------+------------------+
|summary|          product_id|product_category_name|product_name_lenght|product_description_lenght|product_photos_qty|  product_weight_g| product_length_cm| product_height_cm|  product_width_cm|
+-------+--------------------+---------------------+-------------------+--------------------------+------------------+------------------+------------------+------------------+------------------+
|  count|               32951|                32341|              32341|                     32341|             32341|             32949|             32949|             32949|             32949|
|   mean|                null|                 null|  48.47694876472589|         771.4952846232337|2.1889861166939797|2276.4724877841513| 30.81507784758263|16.937661234028347|23.196728277034204|
| stddev|                

## Group by query

In [13]:
products_data.groupBy("product_category_name").count().show()

+---------------------+-----+
|product_category_name|count|
+---------------------+-----+
|                  pcs|   30|
|                bebes|  919|
|                artes|   55|
|            cine_foto|   28|
|     moveis_decoracao| 2657|
|             pc_gamer|    3|
| construcao_ferram...|  400|
| tablets_impressao...|    9|
| fashion_roupa_mas...|   95|
|    artigos_de_festas|   26|
|     artigos_de_natal|   65|
|           la_cuisine|   10|
|               flores|   14|
|      livros_tecnicos|  123|
|                 null|  610|
|       telefonia_fixa|  116|
| construcao_ferram...|   91|
|           cool_stuff|  789|
|     eletrodomesticos|  370|
|    livros_importados|   31|
+---------------------+-----+
only showing top 20 rows



# Running SQL Queries Programmatically

In [14]:
products_data.createOrReplaceTempView("products_data")

productssql=sc.sql("select product_category_name,count(*) as count from products_data group by product_category_name")

In [15]:
productssql.show()

+---------------------+-----+
|product_category_name|count|
+---------------------+-----+
|                  pcs|   30|
|                bebes|  919|
|                artes|   55|
|            cine_foto|   28|
|     moveis_decoracao| 2657|
|             pc_gamer|    3|
| construcao_ferram...|  400|
| tablets_impressao...|    9|
| fashion_roupa_mas...|   95|
|    artigos_de_festas|   26|
|     artigos_de_natal|   65|
|           la_cuisine|   10|
|               flores|   14|
|      livros_tecnicos|  123|
|                 null|  610|
|       telefonia_fixa|  116|
| construcao_ferram...|   91|
|           cool_stuff|  789|
|     eletrodomesticos|  370|
|    livros_importados|   31|
+---------------------+-----+
only showing top 20 rows



# Converting to RDD

In [16]:
products_rdd=products_data.rdd


In [17]:
#converting df into a rdd of string
products_data.toJSON().first()

'{"product_id":"1e9e8ef04dbcff4541ed26657ea517e5","product_category_name":"perfumaria","product_name_lenght":"40","product_description_lenght":"287","product_photos_qty":"1","product_weight_g":"225","product_length_cm":"16","product_height_cm":"10","product_width_cm":"14"}'

In [19]:
products_data.select("product_category_name").write.json("../data/products_data3.json")

In [ ]:
products_data.select("product_category_name").write.save("../data/products_data.json",format="json")

Py4JJavaError: An error occurred while calling o269.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:638)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:278)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:98)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$6(SQLExecution.scala:109)
	